# <center>Structural Analysis and Visualization of Networks</center>

## <center>Home Assignment #3: Centralities and Assortativity</center>

### <center>Student: *Nazarov Ivan*</center>

#### <hr /> General Information

**Due Date:** 09.03.2015 23:59 <br \>
**Late submission policy:** -0.2 points per day <br \>


Please send your reports to <mailto:leonid.e.zhukov@gmail.com> and <mailto:shestakoffandrey@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *Nazarov* *Ivan* HA*3* **

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Preamble
Import the usual toolbox

In [ ]:
## Comupte the page rank on the row sparse matrix M with the teleportation probability \beta
import numpy as np
import numpy.random as rnd
import scipy.sparse as spma
import scipy.sparse.linalg as spla


import matplotlib.pyplot as plt
%matplotlib inline

## Problems

### Task 1

In order to compute nontrivial cenrtality measures it is useful to represent a sparse connectivity matrix in an adjacency list format.

In [ ]:
from collections import defaultdict
def sparse_adjacency( A, symmetrize = False ) :
    """Requires A to be in either csr, csc or lil format."""
## Construct the adjacency list: get all nonzero indices
    if symmetrize :
        S = A.transpose(copy = True)
        A = S + A
        del S
    r, c = A.nonzero( )
    i, j = np.unique( r, return_index = True )
## Initialize the dictionary and the index
    J = dict( ) ; i0 = 0
## Exploit the fact that the output of nonzero() is sorted lexicographically 
##  and that the np.unique() returns the index of the furst occurrence of a value
    for x, y in zip( i[:-1], j[1:] ) :
## the indices returned by np.unique() are shifted forward to make it
##  easier to capture complete ranges of colunm indices of A
        J[ x ] = c[ i0 : y ]
        i0 = y
## Finalize by capturing the last row
    J[i[-1]] = c[i0:]
## The default value is an empty adjacancy list
    return defaultdict( lambda: np.empty( 0, np.int ), J )


## Centrality measures
#### Degree
The degree centrality measure of a node $v\in V$ in graph $G=\big(V, E\big)$ is the sum of all edges incident on it:

$$C_v = \sum_{u\in V} 1_{(v,u)\in E} = \sum_{u\in V} A_{vu} = \delta_v$$

In other words the more 1st-tier (nearest, reachable in one hop) negihbours a vertex has the higher its centrality is. This is a local metric. In order to perform intergraph comparisons, normalised degreee centrality is used:

$$\bar{C}_v = \frac{C_v}{|V|-1}$$

As usual high centrality -- a node in direct contact with many other nodes, low centrality -- a periphrial vertex.

In [ ]:
def degree_centrality( A, norm = False ) :
    delta = np.array( A.sum( axis = 1 ) ).flatten()
    return delta if not norm else delta / ( A.shape[ 1 ] - 1 )
# c_degree( A, True )

#### Betweenness
This measure assesses how important a node is in terms of the global graph connectivity:

$$C_B(v) = \sum_{s\neq v\neq t\in V} \frac{\sigma_{st}(v)}{\sigma_{st}}$$

where $\sigma_{st}(v)$ is the number of **shortest** paths from $s$ to $t$ passing through $v$, while $\sigma_{st}$ is the total number of paths of least legnth connecting $s$ and $t$. 

In hte case of unweighted graphs,  this measure can be computed in $O(|V||E|)$ time with a modified breadth-first-search algorithm, which while calculating all-pairs shortest paths computes the centrality using some clever arithmetic tricks (See Brandes (2001)).

##### References
Brandes, Ulrik (2001). "A faster algorithm for betweenness centrality" (PDF). Journal of Mathematical Sociology 25: 163–177. doi:10.1080/0022250x.2001.9990249

A generic breadth-first-search is given below:

In [ ]:
def bfs_shortest_path( A, s = 0, pi = None, J = None ) :
## Cache deque operations for faster access (Python won't have to make lookups)
	deq = deque.popleft ; enq = deque.extend
## Get the sparse adjacency list
	if J is None :
		J = sparse_adjacency( A, symmetrize = True )
## Initalize the array of shortest paths
	if pi is None :
		pi = np.full( A.shape[ 0 ], np.inf, np.float )
## Initialize with the source vertex
	Q = deque( [ s ] ) ; pi[ s ] = 0
## While the (de)queue is not empty
	while Q :
## get the virst vertes to have been added
		v = deq( Q )
## For each neighbour...
		n = J[ v ]
## Check if it has not been visited before
		n = n[ pi[ n ] == np.inf ]
## Add it to the queue and update its distance from the source
		if len( n ) :
			enq( Q, n )
			pi[ n ] = 1 + pi[ v ]
	return pi

While the Brandes's algorithm is just the extended version, which incementally counts the betweenness centrality. As stated in the paper, in counting other centrality measure could be easily incorporated in it. 

In [ ]:
from collections import deque
def betweenness_centrality( A, J = None, verbose = False ) :
## Cache deque operations for faster access (Python won't have to make lookups)
	deq = deque.popleft ; enq = deque.extend
	pop = deque.pop ; push = deque.append
	append = list.append
## Get the sparse adjacency list
	if J is None :
		J = sparse_adjacency( A, symmetrize = True )
## Initialize the betweenness array
	cb = np.zeros( A.shape[ 0 ], np.float )
	for s in xrange( A.shape[ 0 ] ) :
		if (s%20)==0 : print s
## Initalize the array of shortest paths
		pi = np.full( A.shape[ 0 ], np.inf, np.float ) ; pi[ s ] = 0
## The array which holds the number of shortest paths through a parciular vertex
		sigma = np.zeros( A.shape[ 0 ], np.float ) ; sigma[ s ] = 1
## Initialize queue with the source vertex, stack and predecessors -- empty.
		Q = deque( [ s ] ) ; S = deque( ) ; pred = defaultdict( list )
## While the (de)queue is not empty
		while Q :
## get the first vertex to have been added
			v = deq( Q )
			push( S, v )
## For each neighbour...
			n = J[ v ]
## Check if it has not been visited before
			w = n[ pi[ n ] == np.inf ]
## Add it to the queue and update its distance from the source
			if len( w ) :
## This is the first time the vertex has been seen
				enq( Q, w )
## Initalize its shortest path through s~v-w edge
				pi[ w ] = 1 + pi[ v ]
## Find neighbours with the shortest path having the current vertex v 
			u = n[ pi[ n ] == 1 + pi[ v ] ]
## For all nodes that already have a shortest path through v
			if len( u ) :
##  update their path count and predecessor lists
## The extra number of shortest paths in this case is equal to
##  the number of shortest paths s~v.
				sigma[ u ] += sigma[ v ]
				for i in u :
					append( pred[ i ], v )
## Initalize the pair dependency array
		delta = np.zeros( A.shape[ 0 ], np.float )
## The stack contains vertices in non-increasing order
		while S :
			w = pop( S )
## Update dependency of each predecessor
			v = pred[ w ]
			if len( v ) :
				delta[ v ] += sigma[ v ] * ( 1.0 + delta[ w ] ) / sigma[ w ]
			if w != s :
				cb[ w ] += delta[ w ]
	return cb
# T = spma.csc_matrix( [ [ 0,1,1,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1], [1,0,0,0,0] ] )
# T = spma.csc_matrix( [ [ 0,0,1,0,0 ], [ 0,0,1,0,0 ], [1,1,0,1,1], [ 0,0,1,0,0 ], [ 0,0,1,0,0 ] ] )
# T = spma.csc_matrix( [ [ 0,0,1,0,0 ], [ 0,0,1,0,0 ], [0,0,0,1,1], [ 0,0,0,0,0 ], [ 0,0,0,0,0 ] ] )
# %lprun -f betweenness_centrality betweenness_centrality( A )
# betweenness_centrality( T )

## Page Rank
The basic idea of Page Rank, though recursive, is to assign each vertex some importance score based on the aggregate importance score of the node connected to in via incoming edges.

The PageRank score has the following interpretation: the more likely it is to find a random surfer in a node, the higher is the pagerank of that node. A random surfer is an agent who follows outgoing links randomly. In terms of the random walk process it is the share of time spent at any particular node in the network.

Immediately two issues become apparent:
 * _Dangling vertices_: the vertices with incoming edges only.  
   Since the network is finte, the pure random walk process would end up in such a sink node with probility one in finite number of steps; To avoid this, the walker is forced to restart at any node in the graph chosen uniformly at random.
 * _Clique traps_. This is a generalization of sink vertices:  
   once in a completely connected subgraph, the random surfer is unable to leave the subgraph. In this case at any given node the process is allowed with some probability to make a jump to an arbitrary vertex in $G$ chosen uniformly at random.

Let $G=(V,E)$ be a directed graph and its incidence matrix be given by $\big(A_{vu}\big)_{u,v\in V}$ where $A_{uv} = 1_{u\leadsto v}$,  ie. the $u,v$-th element indicates the weight in general, or the existence in particular, of an edge $u\leadsto v$ between vertices $u,v\in V$. Let the out degree be defiend as $\delta^+_i = \sum_{j\in V} a_{ij}$.

Consider a $1$-order markov chain with trasnition kernel defined as the stochastic version $P$ of the incidence matrix $A$, defiend as
$$q_{ij} = \frac{ a_{ij} }{\delta^+_i} 1_{\delta^+_i\neq 0} +\frac{1}{|V|} 1_{\delta^+_i = 0}$$
which denotes the probability of a $i\to j$. This transition kernel already incorporates the uniform correction for the dangilng nodes.

Further adjustnet is due. In orther to escape from the clique traps, the trasition distribtuion at each vertex is further smoothed: having selected a particualt edge $i\to j$ to follow, the process actually moves to the destination node $j$ with probability $\beta$ and to any vertex in the graph picked at random (including $j$) with probability $1-\beta$.  

Thus the funal Pagerank process transition kernel is given by the following fomrula for the probability of ending up in one step at node $j$ having started at $i$
$$p_{ij} = \beta q_{ij} + (1-\beta) \frac{1}{|V|}$$

The matrix $p_{ij}$ is in fact row-stochastic, meanin that on each row the values in the columns sum to one. For example if a column vector $(\pi_i)_{i\in V}$ is a prior probability distribution of being in vertex $i$, then the posterior probability is $P'\pi$.

Due to Perron-Frobenius theorem for a well behaved stochastic matrix there necessarily exists a stationary probability distribution. The PageRank is this stationary distribution: $P'\pi = \pi$ with $\pi'\mathbf{1} = 1$. Furhtermore the the largest eigenvalue of $P'$ is exactly $1$, which menas that iterating $\pi_k = P'\pi_{k-1}$ would yield a convergent sequence of probability vectors.

In general the incidence matrices are quite large in terms of dimensions which actually requires some modification of how exactly the next refinenemt should be constructed:
$$\pi_{k+1}' = \beta \pi_k' D^{-1} A + \big( \beta \pi_k'd + 1-\beta\big) \frac{1'}{|V|}$$
where $D_{ii} = \delta^+_i$ is a diagonal matrix. This expression does not require the transpose of the incidence matrix and 

The relative tolerance convergence criterion is
$$\max_{i\in V} \big\lvert \pi_{k+1} - \pi_k\big\rvert \leq \epsilon|\pi_k| + \epsilon^2$$
this permits relative error of approximately $\epsilon$ tolerance for non-zero values of $\pi_k$, and yet requires higher precision if the value on hte previous step was zero. 

#### The iterative procedure
As have been mentioned eralier the PageRank score is the egienvector of a stochastic matrix corresponding to the unit eigenvalue. 

In [ ]:
def PageRank_iter( A, beta = 0.85, x0 = None, rel_eps = 1.0E-8, niter = 10000 ) :
## Create a teleporation vector
    E = np.full( A.shape[ 0 ], 1.0 / A.shape[ 0 ], np.float )
## If the initial ranking is not provided us the uniform distribution over the nodes
    x0 = np.copy( E ) if x0 is None else x0
## Find the normalising constants for each row
    out = np.array( A.sum( axis = 1 ), np.int ).flatten( )
## Locate the dangling vertices
    dan = np.array( out == 0 )
##  ... and reset their normalising constant to avoid NANs
    out[ dan ] = 1.0
## The resulting status of the convergence procedure:
##  0 -- convergence within the set relative tolerance
##  1 -- exceeded the number of iterations.
    status = 1 ; i = 0
## First stopping rule: within the specified number of iterations
    while i < niter :
## The main computational step
        x1 = beta * ( x0 / out ) * A + ( beta * np.sum( x0 * dan ) + 1 - beta ) * E
## Second stopping rule: within the required tolerance. Correction for 
##  possible machine zeros in the denominator.
        if np.sum( np.abs( x1 - x0 ) / ( np.abs( x0 ) + rel_eps ) ) < rel_eps :
            status = 0
            break
## Proceed to the next iteration
        x0 = x1 ; i += 1
## return the stationary distribution and the convergence information
    return ( x1, { 'convergence': status, 'iterations' : i } )
## Some small test cases
# T = spma.csc_matrix( [ [ 0,1,1,0], [0,0,1,0], [1,0,0,1], [0,0,0,0] ] )
# T = spma.csc_matrix( [ [ 0,1,1,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1], [1,0,0,0,0] ] )
# print PageRank_iter( T, .9, rel_eps = 1e-10 )

## Hubs and Authorities index

The basic idea of this ranking is ...

The HITS algorithm preforms the following steps in succession until the convergence criteria are met:
 * the current authority and hub scores are given by $a_i$ and $h_i$ respectively;
 * Based on the current scores calculate the next iteration:
    * $a_{i+1} = \alpha A' h_i$
    * $h_{i+1} = \beta A a_{i+1}$

This implies that $h_{i+1} = \alpha \beta AA' h_i$ which, when re-normalised by its $l_2$-norm, is known to converge to the egienvector of $AA'$ with the largest eigenvalue.
Simalrly, $a_{i+1} = \alpha \beta A'A a_i$ is the main iterative core, which converges to the eigenvector of $A'A$.

The SVD of an $n\times m$ matrix $A$ of rank $k$ is a decomposition of it into the product $U\Sigma V'$ with $U$ -- an $n\times k$ matrix of orthogonal columns, $V$ -- an $m\times k$ matrix again of orthogonal columns, and a diagonal $k\times k$ matrix of positive singular values.
It is easy to see, that $A'A = V\Sigma'\Sigma V'$ and $AA'=U\Sigma\Sigma' U'$, whence $A'A V = V\Sigma'\Sigma$ and $AA'U = U\Sigma\Sigma'$. Therefore the left and the right singular vector are given by the columns of $U$ and $V$ respectively.

The HITS algorithm essentially performs power iterations on the pair of matrices $A'A$ and $AA'$ simultaneously. That is why 
:
 1. the authority and the hub scores are the right and the left signular vectors respectively;
 2. the algorithm converges to a pair of singular vectors corresponding to the largest singular value.


In [ ]:
def HITS_iter( A, xy0 = None, rel_eps = 1.0E-8, niter = 10000 ) :
## Initialise to the uniform hub-authority vector
    if xy0 is None:
        xy0 = np.full( ( 2, A.shape[0] ), 1.0 / np.sqrt( A.shape[0] ), np.float )
## A_{ij} is whether there is an edge i\to j
    status = 1 ; i = 0
    while i < niter :
        xy = xy0.copy( )
## Compute the authorities (a' = \alpha h' A)
        xy[1] = xy[0] * A
        xy[1] /= np.linalg.norm( xy[1] )
## and the hubs score (h = \beta A a)
        xy[0] = A * xy[1]
        xy[0] /= np.linalg.norm( xy[0] )
        if np.sum( np.abs( xy - xy0 ) / ( np.abs( xy0 ) + rel_eps ) ) < rel_eps :
            status = 0
            break
        xy0 = xy ; i += 1
    return ( xy, { 'convergence': status, 'iterations' : i } )
# A = spma.csc_matrix( [ [ 0,1,1,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1], [1,0,0,0,0] ], dtype = np.float )
# ha, _ = HITS_iter( A )
# h, _, a = spma.linalg.svds( A, 1 )
# h = np.abs( np.array( h ).flatten( ) )
# a = np.abs( np.array( a ).flatten( ) )
# print np.sum( np.abs(ha[0]-h)/( np.abs( h ) + 1e-8) )
# print np.sum( np.abs(ha[1]-a)/( np.abs( a ) + 1e-8) )

<hr/>

You can load .mat files with the following commands:

In [ ]:
import scipy.io
data = scipy.io.loadmat('./data/flickr.mat')

A = spma.csc_matrix( data[ 'A' ] )
names = data[ 'names' ]

Data contains sparse matrix A and list of user names.
This is a “denser” part of the Flickr photo sharing site friendship graph from 2006. Edge direction corresponds to friendship requests (following). Some of the links are reciprocal,others not.  

Compute degree centrality, Pagerank and  HubAuthorities scores for the [flickr](https://www.dropbox.com/s/6c4ybirvffrm2tu/flickr.zip?dl=0) network. 
Provide top 50 names in each ranking, compare results

In [ ]:
T = 100

#### Solution
Compute the page rank for different restart-teleport probabilities

In [ ]:
beta = np.arange(0, 1, 0.01 ) ##[ 0.01, .15, .25, .50, .75, .85, .99 ]
pr = [ ( PageRank_iter( A, beta = b ), b ) for b in beta ]

Get the 50 top ranking vertices for each value in the $1\times 100$ $\beta$-grid.

In [ ]:
top_pr_test = [ np.argsort( r )[:-T-1:-1] for (r,_), b in pr ]

The nature of the rankings suggests that the similarity beetween them is calculated in a discrete manner -- using Jaccard's similarity measure.

In [ ]:
def jaccard( a, b ) :
    x,  y = set( a ), set( b )
    return ( len( x & y ) + 0.0 ) / len( x | y )

Use the scipy's spatial module to easily compute the pairwise similarity.

In [ ]:
from scipy.spatial.distance import pdist, squareform
D = squareform(pdist(top_pr_test, jaccard))

Disaplay the similarity of 50 top ranking node in Flickr network between PageRank for ach pair of $\beta$.

In [ ]:
plt.figure( figsize=(8,6) )
plt.title("""Jaccard similarity between PageRank rankings""")
plt.imshow( D )
plt.colorbar()
plt.show()

In general close settings of $\beta$ tend to produce close rankings. This plot clearly shows that, the pageranks for $\beta\in \big[0.375 0.625\big]$ produce constitute the largest spectral cluster of similar rankings. Now display the top

In [ ]:
## Lets compute the pagerank which is more determined
##  by the adjacency matrix rather than the teleportation,
pr, _ = PageRank_iter( A, beta = .85, rel_eps = 1.0E-7)
## Settle on this paramter
top_pr = np.argsort( pr )[:-T-1:-1]

In [ ]:
## Compute the Hubs-Authorities scores
ha, _ = HITS_iter( A )
top_hs = np.argsort( ha[0] )[:-T-1:-1]
top_as = np.argsort( ha[1] )[:-T-1:-1]

In [ ]:
## And finally compute the degree centrality
cd = degree_centrality( A, norm = False )
top_cd = np.argsort( cd )[:-T-1:-1]

Computing the betweenness centrality has $O(n m)$ compelixty, but still this took almost 2 hours of single-threaded computation time. Therefore in order to be able to comapre the results, I decied to store the results for symmetrized Flickr network in an embedded base64-encoded CLOB (**C**haracter **L**arge **OB**ject) cache.

In [ ]:
import base64
import zlib
USE_CACHED = True

if not USE_CACHED :
## Even though the algorithm is sufficiently optimal : O(|V||E|) complexity for a sparse unwweighted matrix
##  still in this single-threaded version it took about 120 minutes to compute the whole vector.
    cb = betweenness_centrality( A )
## Thus after the computation has finished create a CLOB with the base64 encoded compressed computed data
    cb_b64_clob = base64.b64encode( zlib.compress( cb ) )

In [ ]:
if not USE_CACHED :
## Print the computed CLOB
    print cb_b64_clob
if USE_CACHED :
## The cached result of betweennesss computation (remember 2 hours!)
    cb_b64_clob = """..."""
## Reconstruct the precalculated betweenness centrailty for the Flick matrix from the cached CLOB
    cb = np.frombuffer( zlib.decompress( base64.decodestring( cb_b64_clob ) ), dtype = np.float )
top_cb = np.argsort( cb )[:-T-1:-1]

It is not very informative to print large swaths of top ranked names, since there is just too much information. So in order to get quantitative characteristis of differences between top-100 rankings it might be reasonable to use Jaccard similarity coefficient, implemented above:

$$\text{J}(A,B)= \frac{\lvert A\cap B\rvert}{\lvert A\cup B\rvert}$$

It is sometimes interesting to look at a table representation of a symmetric distance matrix. The procedure below prints a matrix in a more straightforward format.

In [ ]:
## Print the upper triangle of a symmetric matrix in reverse column order
def show_symmetric_matrix( A, labels, diag = False ) :
    d = 0 if diag else 1
    c = len( labels ) - d
    print "\t", "\t".join( c * [ "%.3s" ] ) % tuple( labels[ d: ][ ::-1 ] )
    for i, l in enumerate( labels if diag else labels[ :-1 ] ) :
        print ( ( "%4s\t" % l ) + "\t".join( ( c - i ) * [ "%.3f" ] ) %
                tuple( rank_dist[ i,i+d: ][ ::-1 ] ) )

In is interesting to note that Betweenness Centrality and the PageRank produce rankings that match each closer than other methods.

In [ ]:
## Compute the distances
labels = [ 'btw', 'deg', 'hub', 'aut', 'prk' ]
rank_dist = squareform( pdist( [ top_cb, top_cd, top_hs, top_as, top_pr ], jaccard ) )
show_symmetric_matrix( rank_dist, labels )

Furhermore it is interesting to assess how the various raw rankings are aligned. In order to avoid issue with different normalisations of the rank vectors, one can use Spearman's rank correlation, which is invariant under monotonic transformations.

In fact Spearman's $\rho$ is the most direct way to compare the ordering, since in its core it does exactly that: it compares the rank-transformation of one vector of observed data to another.

In [ ]:
from scipy.stats import spearmanr as rho
rank_dist = squareform( pdist( [ cb, cd, ha[0], ha[1], pr ], metric = lambda a, b : rho(a,b)[0] ) )
show_symmetric_matrix( rank_dist, labels )

The rankings produced by different methods are very close. 

### <hr /> Task 2

Here are the [Facebook friendship graphs](https://www.dropbox.com/s/l4z8x81a9bolpw5/universities.zip?dl=0) from several US universities from 2005 (one year after fb launch).

Data contains a A matrix (sparse) and a "local_info" variable, one row per node: 
a student/faculty status flag, gender, major, second major/minor (if applicable), dorm/house, year, and high school. 
Missing data is coded 0.

Compute node degree assortativity (mixing by node degree) and assortativity coefficient (modularity) for gender, major, dormitory, year, high school for all universities and compare the results.

### Toolkit

In order to be able to compute assortativity measures for categorical data efficiently, it would be handy to be able to partition the pertices by the value of a discrete random variable.

In [ ]:
## Compute the inverse image of the variable trated as a generalized map
def group_by( variable, ignore_float_nan = False ) :
    order = np.array( np.argsort( variable ), dtype = np.int )
## NANs, if present, would be concentrated and the end of the value-index pair
##  and in the tail of the order index array.
    value, index = np.unique( variable[ order ], return_index = True )
## It turns out the searchsorted actually does find the first occurrence
##  of a NAN, even though technically NAN != NAN. This is unexpected since
##  for IEEE754 floats NANs force-fail every comparison. 
    first_nan = np.searchsorted( value, np.nan )
    if first_nan < len( value ) :
        if not ignore_float_nan :
            value, index = value[ : first_nan+1 ], index[ : first_nan+1 ]
        else :
            order = order[ :index[ first_nan ] ]
            value, index = value[ : first_nan ], index[ : first_nan ]
    G = dict( ) ; prev = 0
    for x, y in zip( value[:-1], index[1:] ) :
        G[ x ] = np.sort( order[ prev : y ] )
        prev = y
    G[ value[ -1] ] = np.sort( order[ prev: ] )
    return defaultdict( lambda: np.empty( 0, np.int ), G )

Directly computing the modularity coefficient for catergorical data (as well as for scalar) is enormously time consuming. However, the sum

$$\sum_i \sum_j \Big( A_{ij} - \frac{\delta_i\delta_j}{2m}\Big) 1_{c_i=c_j}$$

is equivalent to 

$$\sum_{c\in \mathcal{C}} \Big(  \sum_{i, j\in c^{-1}}A_{ij} - \frac{1}{2m} \sum_{i\in c^{-1}} \delta_j\sum_{j\in c^{-1}}\delta_i\Big)$$

where $\mathcal{C}$ is the set of all posssible values of a discrete (categorical) feature and $c^{-1}$ is the set of all objects $i$ such that $c_i = c$ :
$$c^{-1} = \big\{\big.i\,\rvert\,c_i=c\big.\big\}$$

In [ ]:
def modularity( A, C, J = None, G = None, assortativity = False ) :
## A_{ij} entry corresponds to an indicator if the edge i->j exists.
    if G is None : G = group_by( C )
# A had better be in csr format
    if J is None : J = sparse_adjacency( A, symmetrize = False )
    D = np.array( A.sum( axis = 1 ), dtype = np.float ).flatten( )
    D_sum = np.sum( D, dtype = np.float )
    dep = dict( ) ; indep = dict( )
    for k in G :
        g = G[ k ]
        sum_k = np.sum( D[ g ], dtype = np.float64 ) 
        indep[ k ] = ( sum_k / D_sum ) * sum_k
## Append an guaranteed infinty at the end of the goup
        bins = np.full( g.shape[0]+1, np.inf, dtype = np.float )
        bins[:g.shape[0]] = g
## Since the groups are force-sorted, binary search can used to infer which
##  nodes in the group have the same category. This way a node with id beyond
##  the group would still be found within the bins and correctly handled, as
##  the node ids can never be infinite.
        dep[ k ] = np.sum( [ np.sum( bins[ np.searchsorted( bins, J[ i ] ) ] == J[ i ],
            dtype = np.float64 ) for i in g ], dtype = np.float64 )
    if not assortativity :
        return ( np.sum( dep.values() ) - np.sum( indep.values() ) ) / D_sum
    return ( np.sum( dep.values() ) - np.sum( indep.values() ) ) / ( D_sum - np.sum( indep.values() ) )

Computing assortatitivy coefficient for scalar-valued features is much easier. Indeed, the mixing coefficient for $X = \big(x_i\big)$ in an undirected graph $G$, with the adjacency matrix $A$, is defined as

$$\rho(x) = \frac{\text{cov}}{\text{var}} = \frac{\sum_{ij}A_{ij}(x_i-\bar{x})(x_j-\bar{x})}{\sum_{ij}A_{ij}(x_i-\bar{x})^2}$$

where $\bar{x} = \frac{1}{2m}\sum_i \delta_i x_i$ is the mean value of $X$ weighted by vertex degree. Note that $A$ is necessarily symmetric.

Since $\delta_i = \sum_j A_{ij}$, it is possible to notice that in matrix notation $\bar{x} = (\mathbf{1}'A\mathbf{1})^{-1} A\mathbf{1}X$ where $\mathbf{1} = \big(1_i\big)$ is a vector of $1$. Let $D = A\mathbf{1}$ be the vector of vertex degrees. Now, the expression in the numerator becomes
$$\sum_{ij}A_{ij}(x_i-\bar{x})(x_j-\bar{x}) = (X-\mathbf{1}\bar{x})'A(X-\mathbf{1}\bar{x}) = X'AX - \bar{x}\mathbf{1}'AX - X'A\mathbf{1}\bar{x} + \bar{x}\mathbf{1}'A\mathbf{1}\bar{x}$$
This expression simplifies to $X'AX - \bar{x}\mathbf{1}'A\mathbf{1}\bar{x}$, because 
* $\bar{x}\mathbf{1}'AX = \bar{x}D'X = X'D(\mathbf{1}'A\mathbf{1})^{-1}D'X = X'D\bar{x} = X'A\mathbf{1}\bar{x}$;
* $\bar{x}\mathbf{1}'A\mathbf{1}\bar{x} = X'D(\mathbf{1}'A\mathbf{1})^{-1}\mathbf{1}'A\mathbf{1}\bar{x} = X'D\bar{x}$.

Note that $\mathbf{1}'A\mathbf{1} = \mathbf{1}'\text{diag}(D)\mathbf{1}$.

Furthermore $\mathbf{1}'A\mathbf{1} = 2m$, whence the numerator take the final form of $X'AX - 2m \bar{x}^2$. Similar matrix manipulation yields the following expression for the denominator : $X'\text{diag}(D)X - 2m \bar{x}^2$.

To recover the the original formula, observe that $2m \bar{x}^2 = X'D(\mathbf{1}' A \mathbf{1})^{-1}D'X = X'D(\mathbf{1}' \text{diag}(D) \mathbf{1})^{-1}D'X$, whence:
* $X'AX - 2m \bar{x}^2 = X'\big(A-D(\mathbf{1}' A \mathbf{1})^{-1}D'\big)X$;
* $X'\text{diag}(D)X - 2m \bar{x}^2 = X'\big(A-D(\mathbf{1}' \text{diag}(D) \mathbf{1})^{-1}D'\big)X$.

The assortativity coefficient for scalar values thus becomes:
$$\rho(x)= \frac{X'AX - 2m \bar{x}^2}{X'\text{diag}(D)X - 2m \bar{x}^2} = \Big(\sum_{ij}\big(\delta_i 1_{i=j} - \frac{\delta_i\delta_j}{2m}\big)x_i x_j\Big)^{-1}\sum_{ij}\big(A_{ij} - \frac{\delta_i\delta_j}{2m}\big)x_i x_j $$

In [ ]:
def assortativity( A, x ) :
## Compute the x'Ax part
    xAx = np.dot( x, np.array( A.dot( x ) ).flatten( ) )
##  and the x'\text{diag}(D)x part. Note that left-multiplying a vector
##  by a diagonal matrix is equivalent to element-wise multiplication.
    D = np.array( A.sum( axis = 1 ), dtype = np.float ).flatten( )
    xDx = np.dot( x, np.multiply( D, x ) )
## numpy.average() actually normalizes the weights.
    x_bar = np.average( x, weights = D )
    D_sum = np.sum( D, dtype = np.float )
    return ( xAx - D_sum * x_bar * x_bar ) / ( xDx - D_sum * x_bar * x_bar )
# D = np.array( conn.sum( axis = 1 ), dtype = np.float ).flatten( )
# print assortativity( conn, D )
# S_e = np.dot( D, np.array( conn.dot( D ) ).flatten( ) )
# S_3 = np.dot( D, np.multiply( D, D ) )
# S_2 = np.dot( D, D )
# S_1 = np.sum( D, dtype=np.float64 )
# print (S_e*S_1-S_2*S_2)/(S_3*S_1-S_2*S_2)

### Preliminary analysis

First, a handy procedure for loading Facebook data an preprocessing it.

In [ ]:
import scipy.io
def loadmat( name, preprocess = False ) :
    data = scipy.io.loadmat( name )
## Read the boolean connectivity matrix and the associated data
    A = spma.csr_matrix( data[ 'A' ], dtype = np.bool )
## student/faculty status flag, gender, major, second major/minor (if applicable), dorm/house, year, and high school
##  HH = { 'status':0, 'gender':1, 'major':2, 'minor':3, 'dorm':4, 'grad':5, 'school':6 }
    info = np.array( data[ 'local_info' ], dtype = np.float)
    if preprocess :
        info[ info == 0 ] = np.nan
## Preprocess graduation year. Convert occasional 1900 to NAN and
##  pool all graduation years prior to 2002 (including) together
##  into a single '≤2002'group.
        info[ info[:,5] <= 1900, 5] = np.nan
        info[ info[:,5] <= 2002, 5] = 2002
    return A, info

Before moving on with the analysis it is necessary to mark mising observations. In the case of the "graduation year" it seems logical to treat years prior to 1900 as missing as well and pool everyone, who graduated earlier that 2002 (including), into one single group.

In [ ]:
A, I = loadmat( './data/universities/Texas80.mat' )
g, c = np.unique( I[:, 5], return_counts = True )
print 'Number of people, who graduated prior to 2002: ', np.sum( c[np.logical_and(g!=0, g<=2002)] )
print 'Total number of people with nonmissing graduation year: ', np.sum( c[g!=0] )

These obseravations justify pooling.

For each university let's compute and compare:
* node degree assortativity;
* modularity coefficient for gender, major, dormitory, year and high school.

In [ ]:
datafiles = [
	'Berkeley13.mat', 'Harvard1.mat', 'Princeton12.mat',
	'Caltech36.mat', 'Oklahoma97.mat', 'Texas80.mat' ]
abbreviations = [ 'BERK', 'HARV', 'PRIN', 'CIT', 'OU', 'UT' ]
results = dict()
for name, abbr in zip( datafiles, abbreviations ) :
    A, I = loadmat( './data/universities/' + name, preprocess = True )
    D = np.array( A.sum( axis = 1 ), dtype = np.float ).flatten( )
    results[ name ] = ( str( name ), str( abbr ), assortativity( A, D ), ) + tuple( [
        modularity( A, I[:,j], G = group_by( I[:,j], ignore_float_nan = True), assortativity = True )
## { 'gender':1, 'major':2, 'dorm':4, 'grad':5, 'school':6 }
            for j in [ 0, 1, 2, 4, 5, 6 ] ] )
## Node degree assortativity coefficoent can also be calculated using:
# S_e = np.dot( D, np.array( A.dot( D ) ).flatten( ) )
# S_3 = np.dot( D, np.multiply( D, D ) )
# S_2 = np.dot( D, D )
# S_1 = np.sum( D, dtype=np.float64 )
# (S_e*S_1-S_2*S_2)/(S_3*S_1-S_2*S_2)

Print the table old-school way:

In [ ]:
print "%16s\tABBR\tDEGREE\tSTATUS\tGENDER\tMAJOR\tDORM\tGRAD\tSCHOOL"%"file"
for k in results:
    print "%16s\t%.5s\t%+4.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" % results[k]

The table clearly shows that:
* friendship formation is poorly determined by either gender or the SAT score (MAJOR) across all university communities;
* in all universities people tend to ignore high school when befreinding anyone. However OU and UT stand out relatively to others, maybe this can be explained by the fact that these universities are located in neighbouring southern states;
* in CIT friends tend to live in acommodation of the same type, whereas collectives at PRIN and HARV tend do disregard this;
* across all universities friendships are considerably more frequent within the same graduation year rather than between them. Especially strong sense of community in same-year alumni is noticable in HARV and PRIN;
* the status is a major factor in making acquaintances in HARV and PRIN; in universities of technology (BERK and CIT) this assortativity is less prominent, but in general higher that in souther universities;
* In HARV and UT popular people tend to facebook-friend other popular people more than at other universities, but stll this is not a major factor in making friends.